### 1. Instalación e inicialización de Spark

In [ ]:
# Actualización de los repositorios de UBUNTU
!sudo apt-get update

In [ ]:
# Instalación de JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Descarga de SPARK
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
# Des-zipeado del instalador
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
# Instalación de Spark en Python
!pip install -q findspark

In [ ]:
# Configuración de variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [ ]:
# Inicialización de Spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
# Verificación de la sesión de Spark
spark

### 2. Creación de estructura de directorios para las capas LANDING_TMP y LANDING

In [ ]:
# Librería para manipulación del sistema de archivos
import os

In [ ]:
# Supongamos que vamos a ingestar del servidor "SERVER_CLIENTES_EMPRESAS" dos archivos estructurados
#
# - ENTIDAD CLIENTE
# - ENTIDAD EMPRESA

In [ ]:
# Creamos el directorio asociado al servidor desde donde vamos a ingestar los datos para la capa "LANDING_TMP"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING_TMP/SERVER_CLIENTES_EMPRESAS')

In [ ]:
# Creamos el directorio para "CLIENTE" en "LANDING_TMP"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING_TMP/SERVER_CLIENTES_EMPRESAS/CLIENTE')

In [ ]:
# Creamos el directorio para "EMPRESA" en "LANDING_TMP"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING_TMP/SERVER_CLIENTES_EMPRESAS/EMPRESA')

In [ ]:
# Creamos el directorio asociado al servidor desde donde vamos a ingestar los datos para la capa "LANDING"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS')

In [ ]:
# Creamos el directorio para "CLIENTE" en "LANDING"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/CLIENTE')

In [ ]:
# Creamos el directorio para "EMPRESA" en "LANDING"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/EMPRESA')

### 3. Subir archivos a la capa LANDING_TMP

In [ ]:
# Subimos los datos de CLIENTE (el archivo 'cliente.txt') a "LANDING_TMP/SERVER_CLIENTES_EMPRESAS/CLIENTE"

In [ ]:
# Subimos los datos de EMPRESA (el archivo 'empresa.txt') a "LANDING_TMP/SERVER_CLIENTES_EMPRESAS/EMPRESA"

### 4. Utilizamos Spark para leer datos de la capa LANDING_TMP como dataframe  

In [ ]:
# Como son datos estructurados nos conviene verlos como tablas
dfCliente = spark.read.format("csv").option("header","true").option("delimiter","|").load("/content/drive/MyDrive/DATALAKE/LANDING_TMP/SERVER_CLIENTES_EMPRESAS/CLIENTE")

# Mostramos los datos
dfCliente.show()

# SIEMPRE y SIN EXCEPCIÓN la zona de LANDING_TMP y LANDING tienen que tener todos los campos del tipo STRING
dfCliente.printSchema()

+---+---------+--------------+--------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+--------------------------------------+-------------+----+-------+----------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                |2004-04-23   |32  |20095  |5         |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu|2019-02-17   |34  |9298   |2         |
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk              |2002-08-01   |27  |10853  |3         |
|4  |Aidan    |1-719-862-9385|euismod.et.commodo@nibhlaciniaorci.edu|2018-11-06   |29  |3387   |10        |
|5  |Leandra  |839-8044      |at@pretiumetrutrum.com                |2002-10-10   |41  |22102  |1         |
+---+---------+--------------+--------------------------------------+-------------+----+-------+----------+
only showing top 5 rows

roo

In [ ]:
# Como son datos estructurados nos conviene verlos como tablas
dfEmpresa = spark.read.format("csv").option("header","true").option("delimiter","|").load("/content/drive/MyDrive/DATALAKE/LANDING_TMP/SERVER_CLIENTES_EMPRESAS/EMPRESA")

# Mostramos los datos
dfEmpresa.show()

# SIEMPRE y SIN EXCEPCIÓN la zona de LANDING_TMP y LANDING tienen que tener todos los campos del tipo STRING
dfEmpresa.printSchema()

+---+---------+
|ID |NOMBRE   |
+---+---------+
|1  |Walmart  |
|2  |Microsoft|
|3  |Apple    |
|4  |Toyota   |
|5  |Amazon   |
|6  |Google   |
|7  |Samsung  |
|8  |HP       |
|9  |IBM      |
|10 |Sony     |
+---+---------+

root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)



### 5. Utilizamos Spark para escribir datos de la capa LANDING_TMP en la capa LANDING

In [ ]:
# Realizamos la binarización de los datos de "CLIENTE" y lo guardamos en el directorio CLIENTE en la capa LANDING
# Creamos un subdirectorio de partición para la fecha '2018-04-19' 
dfCliente.write.format("parquet").save("/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/CLIENTE/2018-04-19")

In [ ]:
# Existe algo llamado RETROCESO en los entornos de DATA LAKE. ¿Qué es un retroceso? lo voy a poner en términos muy simples, al día 
# siguiente viene negocio y dice: “ … oye cometimos un error, el archivo que te pasamos de “clientes” no era el archivo correcto, era otro 
# archivo, entonces, hay que corregir eso, hay que reprocesar … “. ¿Reprocesar que implica? volver a ejecutar el código y que se reemplace el 
# archivo binarizado que vamos a escribir en el directorio: /content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/CLIENTE/2018-04-19.  
# ENTONCES, PARA QUE EL CÓDIGO SEA REPROCESABLE SIEMPRE HAY QUE ACTIVAR EL MODO DE SOBREESCRITURA. Así que, si alguien quiere volver a reprocesar 
# los datos que hemos capturado para cierto día, el código está listo para eso. 
dfCliente.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/CLIENTE/2018-04-19")

In [ ]:
# Escribimos los datos binarizándolos en la partición de la fecha correspondiente en "LANDING" para "EMPRESA"
dfEmpresa.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/EMPRESA/2018-04-19")